### populate

In [1]:
from IPython.core.display import HTML
css = open('../data/style-table.css').read() + open('../data/style-notebook.css').read()
HTML('../data/<style>{}</style>'.format(css))

In [2]:
import datetime, os


from openpyxl import Workbook
from openpyxl.drawing.image import Image
from openpyxl.utils.dataframe import dataframe_to_rows

from journalfiles import JournalFiles
import pandas as pd
from structjour.pandasutil import DataFrameUtil, InputDataFrame, ToCSV_Ticket as Ticket
from structjour.tradeutil import ReqCol, FinReqCol, XLImage, TradeUtil
from withstyle.thetradeobject import SumReqFields, TheTradeObject
from withstyle.tradestyle import TradeFormat
#TradeUtil, FinReqCol, ReqCol, 

hello dataframe


In [3]:
from withstyle.tradestyle import c as tcell

In [4]:
# jf = JournalFiles(theDate = datetime.date(2018, 9, 7), outdir = 'out', mydevel = True)
jf = JournalFiles(outdir='out/', mydevel = True)
# jf=JournalFiles(outdir='out/', mydevel=True)
jf._printValues()

indir:              C:\trader\journal\_09_September\Week_4\_0926_Wednesday
infile:             trades.csv
inpathfile:         C:\trader\journal\_09_September\Week_4\_0926_Wednesday\trades.csv

outdir:             out/
outfile:            Trades_Wednesday_0926.xlsx
outpathfile:        out/Trades_Wednesday_0926.xlsx
theDate:            Wednesday, September 26, 18


In [5]:
        
tkt = Ticket(jf)

tu = TradeUtil()
trades, jf =tkt.newDFSingleTxPerTicket()
# trades = pd.read_csv(jf.inpathfile)

idf = InputDataFrame()
reqCol = ReqCol()
finalReqCol = FinReqCol()

DataFrameUtil.checkRequiredInputFields(trades, reqCol.columns)
trades = idf.zeroPadTimeStr(trades)
trades = trades.sort_values([reqCol.acct, reqCol.ticker, reqCol.time])
trades = idf.mkShortsNegative(trades)
swingTrade = idf.getOvernightTrades(trades)
swingTrade = idf.figureOvernightTransactions(trades)
trades = idf.insertOvernightRow(trades,swingTrade)

trades = tu.addFinReqCol(trades)
newTrades = trades[finalReqCol.columns]
newTrades.copy()
nt = newTrades.sort_values([finalReqCol.ticker,finalReqCol.acct,  finalReqCol.time])
nt = tu.writeShareBalance(nt)
nt = tu.addStartTime(nt)
nt = nt.sort_values([finalReqCol.start, finalReqCol.acct, finalReqCol.time])
nt = tu.addTradeIndex(nt)
nt = tu.addTradePL(nt)
nt = tu.addTradeDuration(nt)
nt = tu.addTradeName(nt)
nt=DataFrameUtil.addRows(nt,1)
nt = tu.addSummaryPL(nt)
    
ldf=tu.getTradeList(nt)

dframe = DataFrameUtil.addRows(nt, 2)



print (ldf[0].Duration.unique()[-1])

topMargin = 10
newdf = DataFrameUtil.createDf(dframe,  topMargin)
insertsize = 25
dframe = newdf.append(dframe, ignore_index = True)

#Add rows and append each trade, leaving space for an image. Create a list of names and rows numbers 
# to place images within the excel file.
imageLocation = list()
for tdf in ldf :
    imageName='{0}_{1}_{2}_{3}.jpeg'.format (tdf[finalReqCol.tix].unique()[-1].replace(' ',''), 
           tdf[finalReqCol.name].unique()[-1].replace(' ','-'),
           tdf[finalReqCol.start].unique()[-1],
           tdf[finalReqCol.dur].unique()[-1])

    # TODO handle empty string in the tdf
    imageLocation.append([len(tdf) + len(dframe) + 2, 
                          tdf.Tindex.unique()[0].replace(' ', '') + '.jpeg',
                          imageName,
                          tdf.Start.unique()[-1],
                        tdf.Duration.unique()[-1]])
    print(len(tdf) + len(dframe) + 2)

    dframe = dframe.append(tdf, ignore_index = True)
    dframe = DataFrameUtil.addRows(dframe, insertsize)
    print(len(dframe))

nt = dframe

before last loop 20
after last loop 24
{'ticker': 'SQ', 'shares': 416, 'before': 0, 'after': 0, 'acct': 'U2429974'}
There is an unbalanced amount of shares of SQ in the amount of 416 
                    in the account U2429974. How many shares of SQ are you holding now? 
                    (Enter for 416) 
That works.
Got SQ with the balance 0, before 0 and after 416 in U2429974
Are we good?
10
Last col? 251.28

            Some shares are unaccounted for. Please send the original csv file to the developer in 
            order to fix ths issue in the software. Please remove the account number  or change its value
            to anything else.
            
Got 10 trades
0:00:18
42
65
69
92
96
119
125
148
152
175
180
203
207
230
234
257
262
285
290
313


In [6]:
wb = Workbook()
ws = wb.active

for r in dataframe_to_rows(nt, index=False, header=False):
    ws.append(r)


for name, cell  in zip(nt.columns, ws[topMargin]) :
    cell.value = name




XL = XLImage()

srf = SumReqFields()
tradeSummaries = list()
tf = TradeFormat(wb)
assert (len(ldf) == len(imageLocation))
     
response = input("Would you like to enter strategy names, targets and stops?")
interview = True if response.lower().startswith('y') else False

Would you like to enter strategy names, targets and stops?y


### Here begins the workspace

In [7]:
loc = imageLocation[0]
tdf = ldf[0]

#     print('Copy an image into the clipboard for {0} beginning {1}, and lasting {2}'.format(loc[1], loc[2], loc[3]))
img = XL.getAndResizeImage(loc[2], jf.outdir)
cellname = 'J' + str(loc[0])
ws.add_image(img, cellname)

#Put together the summary info and interview the trader
tto=TheTradeObject(tdf, interview)
tto.runSummary()
tradeSummaries.append(tto)

#Place the format shapes/styles in the worksheet
tf.formatTrade(ws, anchor=(1, loc[0]))
    
print("Done with interview")


            Copy an image to the clipboard using snip for Trade1_KBH-Short_09:36:22_0:00:18.jpeg
             Are you ready? (q to skip image) y
Side =  SS
What was the strategy?
     1. ORB
     2. ABCD
     3. VWAP Reversal
     4. Bull Flag
     5. Fallen Angel
     6. VWAP False Breakout
     7. VWAP Reversal
     8. 15 Minute Reversal
     9. VWAP MA trend
     10. Other
     11. Skip
1
What was your target? 25.25
What was your Stop Loss? 25.80
Done with interview


In [ ]:
for key in srf.tfcolumns.keys()  :
    cell = srf.tfcolumns[key][0]
    if isinstance(cell, list) :
        cell = cell[0]
    tradeval = tto.TheTrade[key].unique()[0]
    if not tradeval :
        continue
#     print ("{0:10} \t{3} \t{1:}\t{2} ".format(key, cell, tradeval, tcell(cell, anchor=(1, loc[0]))))
    

    ws[tcell(cell, anchor=(1, loc[0]))] = tradeval
    
  
    

### Here ends the workspace

In [8]:
for loc, tdf in zip(imageLocation, ldf) :
#     print('Copy an image into the clipboard for {0} beginning {1}, and lasting {2}'.format(loc[1], loc[2], loc[3]))
    img = XL.getAndResizeImage(loc[2], jf.outdir)
    cellname = 'J' + str(loc[0])
    ws.add_image(img, cellname)
    
    #Put together the summary info and interview the trader
    tto=TheTradeObject(tdf, interview)
    tto.runSummary()
    tradeSummaries.append(tto)
    
    #Place the format shapes/styles in the worksheet
    tf.formatTrade(ws, anchor=(1, loc[0]))
    
    for key in srf.tfcolumns.keys()  :
        cell = srf.tfcolumns[key][0]
        if isinstance(cell, list) :
            cell = cell[0]
        tradeval = tto.TheTrade[key].unique()[0]
        if not tradeval :
            continue
    #     print ("{0:10} \t{3} \t{1:}\t{2} ".format(key, cell, tradeval, tcell(cell, anchor=(1, loc[0]))))


        ws[tcell(cell, anchor=(1, loc[0]))] = tradeval
    
print("Done with interview")


            Copy an image to the clipboard using snip for Trade1_KBH-Short_09:36:22_0:00:18.jpeg
             Are you ready? (q to skip image) y
Side =  SS
What was the strategy?
     1. ORB
     2. ABCD
     3. VWAP Reversal
     4. Bull Flag
     5. Fallen Angel
     6. VWAP False Breakout
     7. VWAP Reversal
     8. 15 Minute Reversal
     9. VWAP MA trend
     10. Other
     11. Skip
1
What was your target? 25.25
What was your Stop Loss? 25.80

            Copy an image to the clipboard using snip for Trade2_CRON-Short_09:40:22_0:02:42.jpeg
             Are you ready? (q to skip image) y
Side =  SS
What was the strategy?
     1. ORB
     2. ABCD
     3. VWAP Reversal
     4. Bull Flag
     5. Fallen Angel
     6. VWAP False Breakout
     7. VWAP Reversal
     8. 15 Minute Reversal
     9. VWAP MA trend
     10. Other
     11. Skip
1
What was your target? 10.85
What was your Stop Loss? 11.17

            Copy an image to the clipboard using snip for Trade3_CRON-Long_09:45:08_0:07

In [9]:
jf.mkOutdir() 
saveName=jf.outpathfile
count=1
while True :
    try :
        wb.save(saveName)
    except PermissionError as ex :
        print(ex)
        print("Failed to create file {0}.{1}".format(saveName, ex))
        print("Images from the clipboard were saved  in {0}".format(jf.outdir))
        (nm, ext) = os.path.splitext(jf.outpathfile)
        saveName = "{0}({1}){2}".format(nm,count,ext)
        print("Will try to save as {0}".format(saveName))
        count=count+1
        if count==6:
            print("Giving up. PermissionError")
            raise (PermissionError("Failed to create file {0}".format(saveName)))
        continue
    except Exception as ex:
        print (ex)
    break
print("Done!")

Done!
